In [1]:
#подгружаем библеотеки Python для работы с языко и random
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

#подгружаем библеотеки торча
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
#Используем CUDу
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
!pip install torchtext

<h2>Будем пилить переводчик с английского на французский, в планах сделать ещё с английского на русский

<h3> Делать будем по этому гайду: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html# , потому будут включены шаги, в которых нет необходимости. В переводчике с английского на русский их мы делать не будем

<h3>Сперва будем использовать кодировку слов при помощи One-hot. Для этого создадим класс содержащий словари слов с соответсвтующими индеками

In [3]:
#Эти токены означают (Start-of-string) и (end-of-string) соответственно, их мы в последующем будем использовать в декодере
#Конкретно SOS мы скармливаем декодеру в начале его работы вместе с contextом Инкодера
SOS_token = 0
EOS_token = 1

class Lang: #классы представляют собой языки с числом тех или иных слов и их индексами
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {} 
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
    # Дополнить список предложением
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1 #Ведём подсчёт того, сколько раз слово встретилось
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1#Ведём подсчёт того, сколько раз слово встретилось

<h3> Создадим функции по предобработке данных: переведём их из Unicode в ASCII, тем самым убрав всякие страшные символы, преобразовав их к более простым, а также уменьшив объём требуемой памяти в два раза, уберём знаки препинания и верхний регистр

In [4]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

<h3> определим функцию чтения файла, которая будет включать в себя обработку строк и индексирование слов

In [5]:
def readLangs(lang1, lang2, reverse=False):# Дэ факто lang1 и lang2 не являются необходимыми параметрами при условии что мы обрабатываем 1 файл
    print("Чтение строк...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse: # на случай если мы хотим сделать не англо-французский переводчик, а французско-английский
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

<h3> дабы быстро что-то обучить, мы возьмём только короткие предложение (не более 10 слов), с простыми конструкциями по типу "я есть ..."

In [6]:
MAX_LENGTH = 100

eng_prefixes = ( #у нас имются 2 варианта на каждый случай по причине того, что мы предврительно удалили апострофы
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

<h3> теперь поместим созданные функции в одну, которая осуществляет полную обработку входящих данных

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Прочитано %s пар предложений" % len(pairs))
    pairs = filterPairs(pairs)
    print("Отобрано %s пар предложений" % len(pairs))
    print("Подсчёт слов...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Число слов:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print("\nПример пары предложений:\n",random.choice(pairs))

Чтение строк...
Прочитано 135842 пар предложений
Отобрано 13078 пар предложений
Подсчёт слов...
Число слов:
fra 5173
eng 3388

Пример пары предложений:
 ['vous etes soucieuse n est ce pas ?', 'you re worried aren t you ?']


<h3>Настало время занятся моделью, для начала инициализируем Encoder. Будем использовать GRU

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)#На вход отправляется вектора размера input_size, которые мы кодируем в вектора размера hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)# Инициализируем модель GRU (Gated Reccurent Unit, она имеет два типа воротЖ)
                                                                   # обновления и сброса, в отличие от 3 в LSTM), которая будет принимать вектора
                                                                    # размера hidden_size и имеет hidden_size скрытых слоёв
                                                                    #, параметр batch_first отвечает лишь за порядок вывода

        self.dropout = nn.Dropout(dropout_p)  # модуль зануляющий каждый элемент тензора с вероятностью dropout_p, что благотовроно влияет на обучение
                                            # нейронной сети, путём предупреждения совместной адаптации нейрнов (Об этом стоит почитать побольше)
    def forward(self, input):
        embedded = self.dropout(self.embedding(input))  # Прогон модели выглядит достаточно просто: имбедим(кодируем), зануляем часть,
        output, hidden = self.gru(embedded)            # пропускаем через GRU, которая на выходе даёт нам набор закодированных предложений и
        return output, hidden                          # конечное скрытое состояние (context)

<h3>Теперь займёмся Decoder-ом. Также будем использовать GRU

In [9]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size) #На вход отправляется output_size слов, которые мы кодируем в вектора размера hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True) # Модель аналогичная по параметрам gru в кодировщике
        self.out = nn.Linear(hidden_size, output_size) #линейное преобразование векторов размера hidden_size, в вектора размера output_size

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None): #на вход передаётся вызод encoder-а и его скрытое состояние (context)
        batch_size = encoder_outputs.size(0) # определяем размер батча
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token) # создаём токен начала предложения
                                                                                                    # для каждгого предложения в батче
        decoder_hidden = encoder_hidden #перенимаем скрытое состояение encoder-а как первоначальное скрытое состояние decoder-a
        decoder_outputs = []

        for i in range(MAX_LENGTH): #делаем до 10 шагов decoder-а
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input // целевой тензор используется для обучения модели
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input // в противном случае используем предыдущие предсказание decoder-а
                # как скрытое состояние
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input??????????????????
        
        decoder_outputs = torch.cat(decoder_outputs, dim=1) #конкатенируем (склеиваем) всё в один тензор (в строках тензора хранится вероятность
                                                            # соответсвующих слов на каждой позиции соответсвенно)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)#при помощи softmax выделяем самое вероятное слово
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop
                        
    def forward_step(self, input, hidden):      # прогон одного шага достаточно прост: имбедим, прогоняем через relu (if x<=0 =>f(x)=0 else f(x)=x),
        output = self.embedding(input)          #далее работает GRU,возвращая свои предсказания и своё скрытое состояние,
        output = F.relu(output)                 # предсказания преобразуем через линейную функцию до требуемого размера
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

<h3>Также создадим декодер с attention, для вычисления оценок внимания будет использоваться механизм Богданау, по факту являющийся моделью прямого распространения

In [10]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size): # при инициализации мы задаём какого размера вектора состояния мы имеем
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size) # инициализируем 3 линейных преобразования, 2 из которых не меняют размерность
        self.Ua = nn.Linear(hidden_size, hidden_size) # а третий превращает матрицу в вектор
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):# в keys попадают скрытые состояния encoder-а на каждом этапе, query - это скрытое состояние decoder-а
                                                                                                                        # в текущий момент
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys))) # сперва строка Wa(querry) прибавляеся к каждой строке Ua(keys), далее
                                                                # примменятеся функция тангенса и всё преобразуется в 1 вектор
        scores = scores.squeeze(2).unsqueeze(1) #сжимаем в третьей позиции и расширяем во второй

        weights = F.softmax(scores, dim=-1) #применяем softmax к каждой строке тем самым выделяя какие состояния мы будем применять 
                                            #на разных шагах и в каком соотношении
        context = torch.bmm(weights, keys) #производится пакетное произведение матриц в тензорах (b,a,c) @ (b,c,d) = (b,a,d)
                                           # таким образом получаем контексты для декодера, являющиеся суммой состояний инкодера в
                                           # разных соотношениях

        return context, weights

In [11]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1): #помимо параметров простого декодера, этот ткже принимает dropout_p
        super(AttnDecoderRNN, self).__init__()                   # чтобы занулять чаасть элементов тензоров
        self.embedding = nn.Embedding(output_size, hidden_size)#На вход отправляется output_size слов, которые мы кодируем в вектора размера hidden_size
        self.attention = BahdanauAttention(hidden_size) # инициализируем наш механизм внимания
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True) #так как в гру будет поступать конкатенация вектора состояния декодера
                                                                          #и контекста полученного при помощи attention, то входной размер в два
                                                                           #раза больше
        self.out = nn.Linear(hidden_size, output_size)#линейное преобразование векторов размера hidden_size, в вектора размера output_size
        self.dropout = nn.Dropout(dropout_p) #модуль зануляющий каждый элемент тензора с вероятностью dropout_p, что благотовроно влияет на обучение
                                            # нейронной сети, путём предупреждения совместной адаптации нейрнов

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None): #На вход берутся состояния инкодера на каждом шагу, конечное сотсояние,
                                                                        #а также возможно, целевой тензор
        batch_size = encoder_outputs.size(0) # определяем размер батча
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)# создаём токен начала предложения
                                                                                                    # для каждгого предложения в батче
        decoder_hidden = encoder_hidden #конечное скрытое состояние инкодера - начальное для декодера
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):  #делаем до 10 шагов decoder-а
            decoder_output, decoder_hidden, attn_weights = self.forward_step( #на вход передаётся вход декодера, его скрытое состояние, 
                decoder_input, decoder_hidden, encoder_outputs                # все состояния инкодера
            )
            decoder_outputs.append(decoder_output) #добавляем выход декодера к прочим
            attentions.append(attn_weights) #добавим веса attention  к прочим

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing // целевой тензор используется для обучения модели(он используетяся
                                                                  #в качесиве входного сигнала декодера вместо его предыдущего предсказания)
            else:
                # Without teacher forcing: use its own predictions as the next input // в противном случае используем предыдущие предсказание decoder-а
                # как скрытое состояние
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1) #производим конкатенацию полученного значения с прочими
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1) # применяем softmax для определения использованногоо слова
        attentions = torch.cat(attentions, dim=1) #конкатенируем веса attention

        return decoder_outputs, decoder_hidden, attentions #возвращаем полученные результаты, с состоянием декодера и весами attention


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input)) #вход декодера мы имбедим и частично зануляем

        query = hidden.permute(1, 0, 2) #происходит перестановка значений скрытого состояния декодера для дальнейшего использования в attention
        context, attn_weights = self.attention(query, encoder_outputs) #при помощи attention получаем контекст и веса
        input_gru = torch.cat((embedded, context), dim=2) #подготавливаем конкатенацию имбеженного тензора в конкатенации с контекстом

        output, hidden = self.gru(input_gru, hidden)# передаём конкатенацию в гру
        output = self.out(output)# записываем результат линейного преобразования к нужному размеру

        return output, hidden, attn_weights #возвращаем значения шага: результат на данном этапе(слове), состояние декодера, а также веса attention

<h3>Для обучения модели необходимо дополнительно обработать наши данные: мы должны превратить пары предложений в соответствующие пары тензоров, состоящих из индексов наших слов

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
def indexesFromSentence(lang, sentence): #функция преобразующая предложение в массив индексов слов этого предложения
    return [lang.word2index[word] for word in sentence.split(' ')] #используя созданный нами класс языка

#def SentenceFromIndexes(lang,indexis):
#    return [lang.index2word[index] for index in indexis]
 #   
#def SentenciesFromIndexies(lang,indexies):
#    return [SentenceFromIndexes(lang, index) for index in indexis]

def tensorFromSentence(lang, sentence): #функция преобразующая предложение в тензор, на вход принимает язык и предложение
    indexes = indexesFromSentence(lang, sentence) #сперва преобразуем в индексы
    indexes.append(EOS_token) # добавляем в конце маркер конца предложения
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1) #возвращает тензор, где все индексы записаны в 1 колонку

def tensorsFromPair(pair): #создаёт пару соответствующих друг другу тензоров
    input_tensor = tensorFromSentence(input_lang, pair[0]) #просто пр именяем преобразование к обоим предложениям в контексте их яхыков
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size): #функция создания datalouder  для двух конкретных языков, английского и французского, на вход принимает размер батча
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True) #предобрабатываем наши строки, создаём для них классы языков

    n = len(pairs) #число пар
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32) #создаём обучающие тензоры имеющие в строках списки индексов слов
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32) #пока что они заполены нолями

    for idx, (inp, tgt) in enumerate(pairs): #через цикл перебираем все пары
        inp_ids = indexesFromSentence(input_lang, inp) #из предложений получаем списки индексов
        tgt_ids = indexesFromSentence(output_lang, tgt) 
        inp_ids.append(EOS_token)#добавляем к ним токен конца предложения
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids#Заполняем обучающие тензоры полученными индексами
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    input_train, input_eval, target_train, target_eval = train_test_split(input_ids, target_ids, test_size=0.2, random_state=42)#разбиваем данные на train
    input_val, input_test, target_val, target_test = train_test_split(input_ids, target_ids, test_size=0.5, random_state=42)# test, val
    
    train_data = TensorDataset(torch.LongTensor(input_train).to(device), 
                               torch.LongTensor(target_train).to(device)) #преобразуем тензоры в датасет, где эти тензоры будут являтся столбцами датасета

    train_sampler = RandomSampler(train_data) #Если я праавильно понял, то мы рандомно дробим наш датасет и согласно этомк разбиению формируем
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size) #наш Datsloader, специальный класс для итерации по 
                                                                                            # датасету

    val_data = TensorDataset(torch.LongTensor(input_val).to(device), 
                               torch.LongTensor(target_val).to(device)) 

    val_sampler = RandomSampler(train_data)                                                   #по отношению к оценочным и тестовым данным действум тем
    val_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)     #же образом
    
    test_data = TensorDataset(torch.LongTensor(input_test).to(device), 
                               torch.LongTensor(target_test).to(device)) 

    test_sampler = RandomSampler(train_data)
    test_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
                                                                                           
    return input_lang, output_lang, train_dataloader, val_dataloader, test_dataloader

<h3> Далее определяем функцию обучения (1 эпоху)

In [14]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer, #на вход передаётся наш dataloader. encoder, decoder, их оптимизаторы
          decoder_optimizer, criterion):                          # а также функция оценки качества (ошибки), которую мы будем минимизировать
    encoder.train()
    decoder.train()#подрубаем режим тренировки
    total_loss = 0 
    for data in dataloader: # производим итерацию по батчам наших данных
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad() #обнуляем градиенты, чтобы они не складывались на каждой итерации
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor) #прогоняем батч через инкодер и декодер
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)), #вычисляем нашу ошибку
            target_tensor.view(-1)
        )
        loss.backward() #проводим обратное распространение, вычисляютя градиенты

        encoder_optimizer.step() #корректируем веса нейронок использую аптимизаторы
        decoder_optimizer.step()

        total_loss += loss.item() #вычисляем ошибку по эпохе (сумма ошибок)

    return total_loss / len(dataloader) #возвращаем среднюю ошибку

<h3> Определим функцию оценки промежуточных результатов

In [15]:
def val_epoch(dataloader, encoder, decoder, #на вход передаётся наш dataloader. encoder, decoder
           criterion):                          # а также функция оценки качества (ошибки), которую мы будем минимизировать
    encoder.eval()
    decoder.eval() #подрубаем режим оценки
    val_loss = 0 
    for data in dataloader: # производим итерацию по батчам наших данных
        
        input_tensor,  target_tensor = data

        encoder_outputs, encoder_hidden = encoder(input_tensor) #прогоняем батч через инкодер и декодер
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)), #вычисляем нашу ошибку
            target_tensor.view(-1)
        )
        val_loss += loss.item() #вычисляем ошибку по эпохе (сумма ошибок)

    return val_loss / len(dataloader) #возвращаем среднюю ошибку

<h3> Создадим вспомогательную функцию для отображения времени и прогресса обучения

In [16]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

 В целом процесс обучения можно представит в следующих этапах <br>
-запуск таймера<br>
-Инициализация оптимизаторов и критерия<br>
-Создание набора обучающих пар<br>
-Запуск пустого массива потерь для построения графа в дальнейшем</p>

Затем мы вызываем много раз train_epoch и печатаем прогресс (% примеров, время на данный момент, расчетное время) и средние потери

In [17]:
def train(train_dataloader,val_dataloader, encoder, decoder, n_epochs, learning_rate=0.001, #передаём наши данные, инкодер, декодер, число эпох
               print_every=100, plot_every=100):          #learning_rate(как сильно мы будем изменять веса каждый раз), а также как часто
                                                        #выводить промежуточные результаты
    start = time.time() #стартуем таймер
    plot_losses = []
    plot_val_losses = []
    print_loss_total = 0  #задаём внутренние переменные
    plot_loss_total = 0  
    print_val_loss_total =0
    plot_val_loss_total =0
    val_loss_previous=math.inf
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate) #инициализируем оптимизаторы для наших нейронок, передаём в них 
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate) # learning_rate
    criterion = nn.NLLLoss() # задаём функцию потерь как для классификации (с этим тоже разобраться стоит подробнее)

    for epoch in range(1, n_epochs + 1): #по количеству эпох итерируем
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion) #тренируем и вычисляем ошибку
        val_loss=val_epoch(val_dataloader,encoder,decoder,criterion)
        print_loss_total += loss
        plot_loss_total += loss#складываем ошибки
        print_val_loss_total += val_loss
        plot_val_loss_total += val_loss#складываем ошибки

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print_val_loss_avg = print_val_loss_total / print_every
            
            print('%s (%d %d%%) train_losses=%.4f val_losses=%.4f' % (timeSince(start, epoch / n_epochs),              #выводим среднюю ошибку на данном этапе
                                        epoch, epoch / n_epochs * 100,print_loss_avg,print_val_loss_avg))
            if (val_loss_previous<=print_val_loss_total):
                print("Training was interupted due to start of overfitting")
                break
            val_loss_previous=print_val_loss_total
            print_val_loss_total = 0
        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg) #складываем для последующего вывода
            plot_loss_total = 0
            plot_val_loss_avg = plot_val_loss_total / plot_every
            plot_val_losses.append(plot_val_loss_avg) #складываем для последующего вывода
            plot_val_loss_total = 0
    print("Training finishid.")
    showPlot(plot_losses) #показываем график ошибок
    showPlot(plot_val_losses)

<h3> Определим функцию вывода ошибок при помощи matplotlib

In [18]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
import matplotlib
plt.close('all')
matplotlib.use('TkAgg')
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()

<h3>Создадим функцию оценки, по большому счёту она похожа на функцию обучения, но использует состояния декодера вместо целевого тензора

In [19]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang): #на вход передаём наши инкодер с декодером, предложение и языки
    with torch.no_grad(): #отключаем расчёт градиента за ненадобностью при оценке
        input_tensor = tensorFromSentence(input_lang, sentence) #преобразуем предложение

        encoder_outputs, encoder_hidden = encoder(input_tensor) # прогоняем его через инкодер и декодер
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)# получаем по 1 одному наибольшему элементу по строкам нашего ответа (по факту самые вероятные слова)
        decoded_ids = topi.squeeze() #сворачиваем в 1 строку

        decoded_words = [] #массив с переводом
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>') #через цикл дополняем наши слова перевода в массив, до того как встретится токен окончания предложения
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn #возвращаем перевод и состояния attention

<h3> создадим функцию для оценки заданного числа предложений, которые мы сможем посмотреть

In [20]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)#рандомно выбираем пару предложений
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang) #производим оценку-перевод
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

<h3> Теперь имея все вспомгательные функции можно начть процесс обучения и оценки

In [21]:
hidden_size = 128
batch_size = 32 #установим 256 скрытых слоёв и размер батча в 32

input_lang, output_lang, train_dataloader,val_dataloader,test_dataloader = get_dataloader(batch_size) #получим данные

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device) #инициализируем наши нейронки
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader,val_dataloader, encoder, decoder, 80, print_every=4, plot_every=5)# начнём обучение

Чтение строк...
Прочитано 135842 пар предложений
Отобрано 13078 пар предложений
Подсчёт слов...
Число слов:
fra 5173
eng 3388
7m 1s (- 133m 21s) (4 5%) train_losses=0.2829 val_losses=0.4005
14m 23s (- 129m 33s) (8 10%) train_losses=0.1472 val_losses=0.3294
21m 27s (- 121m 38s) (12 15%) train_losses=0.1030 val_losses=0.2714
28m 30s (- 114m 1s) (16 20%) train_losses=0.0705 val_losses=0.2052
35m 48s (- 107m 25s) (20 25%) train_losses=0.0478 val_losses=0.1477
42m 36s (- 99m 26s) (24 30%) train_losses=0.0326 val_losses=0.1034
49m 45s (- 92m 25s) (28 35%) train_losses=0.0226 val_losses=0.0692
57m 0s (- 85m 30s) (32 40%) train_losses=0.0162 val_losses=0.0483
63m 56s (- 78m 9s) (36 45%) train_losses=0.0121 val_losses=0.0332
71m 30s (- 71m 30s) (40 50%) train_losses=0.0095 val_losses=0.0239
78m 30s (- 64m 13s) (44 55%) train_losses=0.0077 val_losses=0.0196
86m 4s (- 57m 23s) (48 60%) train_losses=0.0065 val_losses=0.0165
92m 55s (- 50m 1s) (52 65%) train_losses=0.0056 val_losses=0.0139
99m 45s 

<h3> Произведём оценку модели на тестовых данных

In [22]:
criterion = nn.NLLLoss()
val_epoch(test_dataloader, encoder, decoder, criterion)

0.01199440084399929

In [25]:
import torchtext.data.metrics as tm

In [27]:
def bleu_evaluate(test_dataloader,encoder, decoder, input_lang, output_lang): #на вход передаём наши инкодер с декодером, предложение и языки
    with torch.no_grad():
        total_bleu_score=0
        for data in test_dataloader:
            F_indexies,Ideal_translation=data
           
            encoder_outputs, encoder_hidden = encoder(F_indexies) # прогоняем его через инкодер и декодер
            decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden)

            _, topi = decoder_outputs.topk(1)# получаем по 1 одному наибольшему элементу по строкам нашего ответа (по факту самые вероятные слова)
            S_indexies = topi.squeeze() #сворачиваем в 1 строку
          #  print(Ideal_translation)
            decoded_words = [[] for i in range(len(S_indexies))] #массив с переводом
            #print(S_indexies.shape)
            for i in range(len(S_indexies)):
                for idx in S_indexies[i]:
                    if idx.item() == EOS_token:
                        break
                    decoded_words[i].append(output_lang.index2word[idx.item()])
            translation=[[[]] for i in range(len(Ideal_translation))]
           # print( Ideal_translation.shape)
            for i in range(len(Ideal_translation)):
                for idx in Ideal_translation[i]:
                    if idx.item() == EOS_token:
                        break
                    translation[i][0].append(output_lang.index2word[idx.item()])
            #print(decoded_words,"\n",translation)
            bleu_score=tm.bleu_score(decoded_words,translation,max_n = 4)
            total_bleu_score+=bleu_score
            #print(bleu_score)
    
    return total_bleu_score/len(test_dataloader) #возвращаем перевод

In [29]:
bleu_evaluate(test_dataloader,encoder, decoder,input_lang, output_lang)

0.9751919421348755

<h3> Исходя из оценок можно утверждать что мы добились устойчивой работы модели

In [68]:
encoder.eval() #переходим в ознакомительный режим
decoder.eval()
evaluateRandomly(encoder, decoder) # проводим оценку получившегося

> vous n etes pas contusionne
= you re not bruised
< you re not bruised <EOS>

> vous etes probablement trop jeune pour le comprendre
= you re probably too young to understand this
< you re probably too young to understand this <EOS>

> je suis ici pour requerir ton aide
= i m here to ask for your help
< i m here to ask for your help <EOS>

> il est motive pour reussir
= he is eager to succeed
< he is eager to succeed <EOS>

> ils ramassent des noisettes
= they are gathering nuts
< they are gathering nuts <EOS>

> je suis sure que tout ira bien
= i m sure everything will be fine
< i m sure everything will be fine <EOS>

> il n est pas plus fort que moi
= he s not stronger than me
< he s not stronger than me <EOS>

> il a peur de la mer
= he s afraid of the sea
< he s afraid of the sea <EOS>

> vous n etes pas une espionne si ?
= you aren t a spy are you ?
< you aren t a spy are you ? <EOS>

> j ai sommeil alors je pars maintenant
= i m sleepy so i am leaving now
< i m sleepy so i am le

<h3> В целом похоже на правду. Обучение можно считать успешным

<h3> Можно визуализирвать результаты attention, чтобы просмотреть какие именно состояния выбираются attention и влияют на предсказания декодера

In [21]:
import matplotlib
matplotlib.use('TkAgg')
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticks(range(len(input_sentence.split(' '))+2))
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticks(range(len(output_words)+1))
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])


evaluateAndShowAttention('il n est pas aussi grand que son pere')

evaluateAndShowAttention('je suis trop fatigue pour conduire')

evaluateAndShowAttention('je suis desole si c est une question idiote')

evaluateAndShowAttention('je suis reellement fiere de vous')

C:\Users\ov00v\AppData\Local\Temp\ipykernel_4356\2667491203.py:2: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed in 3.10.  To suppress this warning, explicitly call plt.close('all') first.
  matplotlib.use('TkAgg')


input = il n est pas aussi grand que son pere
output = he s not as tall as his father <EOS>
input = je suis trop fatigue pour conduire
output = i m too tired to drive <EOS>
input = je suis desole si c est une question idiote
output = i m sorry if this is a stupid question <EOS>
input = je suis reellement fiere de vous
output = i m really proud of you <EOS>
